In [3]:
from dataclasses import dataclass
import pandas as pd
import os
import SimpleITK as sitk

import keras as tfk
from keras import layers as tfkl

import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.data import Dataset as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

sns.set_theme()

In [4]:
SEED = 42
BATCH_SIZE = 32
IMG_HEIGHT = 512
IMG_WIDTH = 512
SHUFFLE_BUFFER_SIZE = 100
METRICS = ["accuracy", "recall", "f1_score"]

data_dir = '/kaggle/input/lung-ds-norm/Full_slice'

tfk.utils.set_random_seed(SEED)

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=24,            
    #width_shift_range=0.2,        
    #height_shift_range=0.2,       
    shear_range=0.2,              
    zoom_range=0.2,               
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'     
)

val_datagen = ImageDataGenerator(

)

train_generator = train_datagen.flow_from_directory(
    data_dir + "/train",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',           
    #subset='training'               
)

val_generator = val_datagen.flow_from_directory(
    data_dir + "/val",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    #subset='val'
)

Found 1885 images belonging to 5 classes.
Found 475 images belonging to 5 classes.


In [17]:
@dataclass
class Hyperparameters:
    # Model
    activation = "silu"
    # Training
    noise_std: float = 0.05
    optimiser = tfk.optimizers.Adam
    learning_rate = 1e-4
    regularization1 = tfk.regularizers.L1L2(l1=1e-5, l2=0) #1e-4
    regularization2 = tfk.regularizers.L1L2(l1=0, l2=0)
    loss = tfk.losses.CategoricalCrossentropy()
    epochs = 50
    ## Early stopping parameters
    es_patience = 10
    es_min_delta = 1e-2
    ## Learning rate schedule
    lr_patience = 5
    lr_decay_factor = 0.3
    lr_min_delta = 5e-2 
    min_lr = 1e-6

hp = Hyperparameters()

In [18]:
def build_model(hp: Hyperparameters,
                feature_extractor: tfk.applications):
    
    inputs = tfkl.Input((IMG_HEIGHT, IMG_WIDTH, 3))

    gn = tfkl.GaussianNoise(hp.noise_std)(inputs)

    backbone_output = feature_extractor(gn)

    conv_layer = tfkl.Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same')(backbone_output) 
    max_pool_layer = tfkl.MaxPooling2D(pool_size=(2, 2))(conv_layer)
    flatten_layer = tfkl.Flatten()(max_pool_layer) 

    x = tfkl.Dropout(0.6)(flatten_layer)

    x = tfkl.Dense(1024, activation='silu')(x)
    x = tfkl.Dense(512, activation='silu')(x)
    x = tfkl.Dense(256, activation='silu')(x)
    #x = tfkl.Dense(128, activation='silu')(x)

    x = tfkl.Dropout(0.4)(x)

    output = tfkl.Dense(5, activation="softmax")(x)
    
    model = tfk.Model(inputs, output)
    return model

In [9]:
def fit(model: tfk.Model,
        train_generator: ImageDataGenerator,
        val_generator: ImageDataGenerator,
        hp: Hyperparameters):
    model.compile(loss=hp.loss,
                  optimizer=hp.optimiser(learning_rate=hp.learning_rate), 
                  metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=hp.epochs,
        #validation_data=val_dataset,
        callbacks=[
            tfk.callbacks.EarlyStopping(monitor='val_loss', 
                                        #mode='min',
                                        patience=hp.es_patience, 
                                        restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(
                factor=hp.lr_decay_factor,
                patience=hp.lr_patience,
                min_delta=hp.lr_min_delta,
                min_lr=hp.min_lr,
                verbose=1,
            )
    ]
    
    ).history

    return model, history

In [ ]:
backbone = tfk.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(512,512,3),
    pooling=None #'avg'
)

backbone.trainable = False

model = build_model(hp, backbone)

model, history = fit(model, train_generator, val_generator, hp)

Epoch 1/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.1958 - loss: 2.3241 - val_accuracy: 0.2568 - val_loss: 1.5895 - learning_rate: 1.0000e-04
Epoch 2/50
59/59 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.2389 - loss: 1.6218 - val_accuracy: 0.3032 - val_loss: 1.5459 - learning_rate: 1.0000e-04
Epoch 3/50
38/59 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.2592 - loss: 1.5845